In [ ]:
import datetime
x = datetime.datetime.utcnow()
while True:
    if (datetime.datetime.utcnow() - x).seconds == 1:
        x = datetime.datetime.utcnow()
        print(x)
        with open('temp//%s.txt'%str(x),'a') as f:
            f.write(str(x))

In [11]:
from bs4 import BeautifulSoup
import requests
import os, csv
import pandas as pd
import random
import urllib.request

def goback():
    os.chdir('/home/jack/Reject')
    #print(os.getcwd())

In [3]:
df = pd.read_csv('./key.csv')
df.head()

,caseno,Key1,Key2,Unnamed: 3,Unnamed: 4
0,89702625,XpJ13RyT4TGZFVnl2bVFvSWVSeDhldmVtT1F3QT09,XpJ13RyT4U0FYbTBMcXhpV2RYREgxVVkraE9kdz09,NaN,NaN
1,92054897,XpJ13RyT4QXJzM2JyY0NSV09SeDhldmVtT1F3QT09,XpJ13RyT4Z0tNaGFGSEJSTEhTQUZwdzFoYnRxUT09,NaN,NaN
2,89704174,XpJ13RyT4NERudmVETDJaK2lSeDhldmVtT1F3QT09,XpJ13RyT4bm9UT212d1V3Y0JlTDZiZDM0RVM1QT09,NaN,NaN
3,89704176,XpJ13RyT4QStWKzNuK2pRWFdSeDhldmVtT1F3QT09,XpJ13RyT4bm9UT212d1V3Y0RHU0VlelRoYnF1UT09,NaN,NaN
4,89704175,XpJ13RyT4NE9uVkdQNC8rZnlSeDhldmVtT1F3QT09,XpJ13RyT4bm9UT212d1V3Y0JYREgxVVkraE9kdz09,NaN,NaN


In [33]:
casenocode = "XpJ13RyT4TGZFVnl2bVFvSWVSeDhldmVtT1F3QT09"
casenocode2 = "XpJ13RyT4U0FYbTBMcXhpV2RYREgxVVkraE9kdz09"
gethtmlbycodereject(casenocode,casenocode2,'11')
goback()
with open('html//11//11.html', 'r',encoding="utf-8") as f:
    get_Picture(f,'11')
    get_Content(f)

/home/jack/Reject
11
/home/jack/Reject
/home/jack/Reject


In [14]:
def gethtmlbycodereject(casenocode,casenocode2,filename):

    url = "https://twtmsearch.tipo.gov.tw/SS0/friendlyPrinting.jsp?showType=3&caseType=&caseNo=%s&caseNo2=%s&l6=zh_TW&isReadBulletinen_US=&isReadBulletinzh_TW=true"%(casenocode,casenocode2)
    #print(url)
    html = requests.get(url = url)
    soup = BeautifulSoup(html.text, 'html.parser')
    #print(soup.prettify())
    x = html.text
    goback()
    numbers = int(filename) % 40
    #print(numbers)
    os.chdir('html/%s'%str(numbers))
    with open("%s.html"%filename,"w+",encoding="UTF-8") as f:
        f.write(x)

In [15]:
def runhtml(sourcename,failname,startcnt,endcnt):
    
    goback()
    df = pd.read_csv('%s.csv'%sourcename)  

    cnt = 0
    fail = []

    for a in df['caseno']:
        #print(a)
        #print(df['Key1'][df['caseno'] == a])
        #print(df['Key2'][df['caseno'] == a])

        if cnt < startcnt:
            cnt += 1
            continue
        else:
            cnt += 1
            if cnt > endcnt:
                break
            x = df['Key1'][df['caseno'] == a].iloc[0]
            y = df['Key2'][df['caseno'] == a].iloc[0]
            try:
                gethtmlbycodereject(x,y,a)
                #print(x,y,a)
            except:
                print(a)
                fail.append([a,x,y])
                
            goback()
            with open('HTML Progress.txt','a+') as t:
                t.write("HTML Progress: "+str(cnt)+"/ 80597, Failed: "+str(len(fail))+"\n")
            print("HTML Progress: "+str(cnt)+"/ 80597, Failed: "+str(len(fail)))
                
    if len(fail) > 0:
        with open('%s.csv'%failname,'a+' ,newline='') as fd:
            writer = csv.writer(fd)
            writer.writerow(['caseno','Key1','Key2'])
            for k in fail:
                writer.writerow(k)

In [ ]:
goback()
#runhtml('key','failall',5000,80597)
runpics('failpicsall')

In [16]:
def get_Content(html):
    html_text = html.read()
    soup = BeautifulSoup(html_text, 'html.parser')
    #print(soup.prettify())
    td = soup.find_all('td')
    
    for x in td:
        try:
            if 'colTitle' in x.attrs['class']:
                print(x.text)
            else:
                print("\t"+str(x.text))
        except:
            print("")
            print(x.text)
            print("")
            
def get_Picture(html,filename):
    html_text = html.read()
    soup = BeautifulSoup(html_text, 'html.parser')
    image = soup.find_all('div')
    #下面的迴圈是找圖片網址再把結果放到陣列裡
    links=[]
    for d in image:
        if d.find('img'):        #再從div找img裡面的src  
            result=d.find('img')['src']
            links.append(result)
            break

    for link in links:
        if link == "":
            continue
        numbers = int(filename) % 40
        goback()
        os.chdir('pics/%s'%str(numbers))
        local = os.path.join('%s.jpg'%filename)
        full_link = os.path.join('https://twtmsearch.tipo.gov.tw/%s' % link[3:])
        urllib.request.urlretrieve(full_link,local) #link是下載的網址 local是儲存圖片的檔案位址

In [19]:
def runpics(failname):
    
    cnt = 0
    dircnt = 0
    fail = []
    os.chdir('/home/jack/Reject/html')
    dirs = os.listdir()
    
    for j in dirs:
        os.chdir('/home/jack/Reject/html/%s'%str(j))
        files = os.listdir()
        dircnt += 1
       
        for i in files:
            if '.html' not in i:
                continue
            else:
                name = i.split(".")[0]
                #print(name)
                numbers = int(name) % 40
                os.chdir('/home/jack/Reject/html/%s'%str(numbers))
                with open('%s.html'%name, 'r',encoding="utf-8") as f:
                    try:
                        get_Picture(f,name)
                        cnt += 1
                    except:
                        fail.append(name)
                        print(name)
        goback()         
        with open('Pic Progress.txt','a+') as t:
            t.write("Pic Progress: "+str(dircnt)+" / 40; "+str(cnt)+" / 80597, Failed: "+str(len(fail))+"\n")
        print("Pic Progress: "+str(dircnt)+" / 40; "+str(cnt)+" / 80597, Failed: "+str(len(fail))+"\n")
                    
    goback()
    if len(fail) > 0:
        with open('%s.csv'%failname,'a+' ,newline='') as fd:
            writer = csv.writer(fd)
            writer.writerow(['caseno'])
            for k in fail:
                writer.writerow([k])
                

In [8]:
os.mkdir('/home/jack/Reject/html')
os.chdir('/home/jack/Reject/html')
for i in range(40):
    if not os.path.exists(str(i)):
        os.mkdir(str(i))
files = os.listdir()
print(sorted(files))
os.mkdir('/home/jack/Reject/pics')
os.chdir('/home/jack/Reject/pics')
for i in range(40):
    if not os.path.exists(str(i)):
        os.mkdir(str(i))
files = os.listdir()
print(sorted(files))

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '5', '6', '7', '8', '9']
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '5', '6', '7', '8', '9']


In [20]:
os.chdir('/home/jack/Reject/pics')
dirs = os.listdir()
cnt = 0
for i in dirs:
    os.chdir('/home/jack/Reject/pics/%s'%i)
    files = os.listdir()
    cnt += len(files)
print(cnt)
print(80597 - cnt)

80587
10


In [26]:
def get_Content(caseno):
    num = int(caseno % 40)
    os.chdir('/home/jack/Reject/html/%s'%str(num))
    with open('%s.html'%caseno) as html:
        html_text = html.read()
        soup = BeautifulSoup(html_text, 'html.parser')
        print(soup.prettify())
        td = soup.find_all('td')

        for x in td:
            try:
                if 'colTitle' in x.attrs['class']:
                    print(x.text)
                else:
                    print("\t"+str(x.text))
            except:
                print("")
                print(x.text)
                print("")

In [31]:
get_Content(107025361)

<html>
 <head>
  <title>
   錯誤網頁
  </title>
  <title>
   智慧局商標檢索系統
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="-1" http-equiv="Expires">
   <meta content="no-cache" http-equiv="pragma"/>
   <meta content="no-cache" http-equiv="Cache-control"/>
   <meta content="智慧局商標檢索系統" name="keywords"/>
   <link href="/css/default.css;jsessionid=2R5BKWRbne-M9UAT32LlZezElx9mseDPCAJ8KRH1.un082idz" rel="stylesheet" type="text/css"/>
   <link href="/js/jquery-window-5.0.4/css/jquery.window.css;jsessionid=2R5BKWRbne-M9UAT32LlZezElx9mseDPCAJ8KRH1.un082idz" rel="stylesheet" type="text/css"/>
   <link href="/favicon.ico;jsessionid=2R5BKWRbne-M9UAT32LlZezElx9mseDPCAJ8KRH1.un082idz" rel="shortcut icon"/>
   <script src="/jqwidgets/scripts/jquery-1.11.1.min.js;jsessionid=2R5BKWRbne-M9UAT32LlZezElx9mseDPCAJ8KRH1.un082idz">
   </script>
   <script src="/js/tablesort.js;jsessionid=2R5BKWRbne-M9UAT32LlZezElx9mseDPCAJ8KRH1.un082idz" type="text/javascript">
   <

In [2]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import collections
import csv

def get_Content(caseno):
    num = int(caseno % 40)
    os.chdir('/home/jack/2_Reject/html/%s'%str(num))
    with open('%s.html'%caseno) as html:
        html_text = html.read()
        soup = BeautifulSoup(html_text, 'html.parser')
        #print(soup.prettify())
        td = soup.find_all('td')
        #print([x.text for x in td])

        D = collections.OrderedDict()
        A = collections.OrderedDict()
        B = collections.OrderedDict()
        C = collections.OrderedDict()
        L = []
        passlist = ["申請人/商標/標章權人","代理人"]
        alist = []
        blist = []
        clist = []
        
        
        cnt = 0
        loop = 0
        flag = False
        flaga = False
        flagb = False
        flagc = False
        flagaa = False
        flagbb = False
        flagcc = False
        cnta = 1
        cntb = 1
        cntc = 1
        CLASS = -1
        cs = []
        
        for x in td:
            #print(x.text)
            if "代理人" == x.text  or "商標名稱" == x.text :
                flaga = False
            if "商標名稱" == x.text:
                flagb = False
            if "類別" in x.text:
                flagc = False
                
            try:
                if flaga == True:
                    
                    if 'colTitle' in x.attrs['class']:
                        #print("A<-"+x.text)
                        alist.append(x.text)
                        flagaa = False
                    else:
                        #print("\tA->"+str(x.text))
                        flagaa = True

                    if flagaa == True:
                        A["%d申請人/商標/標章權人_"%cnta+alist[-1]] = x.text
                        cnta += 1
                    
                elif flagb == True:
                    
                    if 'colTitle' in x.attrs['class']:
                        #print("B<-"+x.text)
                        blist.append(x.text)
                        flagbb = False
                    else:
                        #print("\tB->"+str(x.text))
                        flagbb = True

                    if flagbb == True:
                        B["%d代理人_"%cntb+blist[-1]] = x.text
                        cntb += 1
                        
                elif flagc == True:

                    if 'colTitle' in x.attrs['class']:
                        #print("C<-"+x.text)
                        clist.append(x.text)
                        flagcc = False
                    else:
                        #print("\tC->"+str(x.text))
                        flagcc = True

                    if flagcc == True:
                        C["%s_類別_"%CLASS+clist[-1]] = x.text
                        cntc += 1
                        
                else:
                    if "類別" in  x.text:
                        #print("<-->"+str(x.text)+"<-->")
                        flagc = True
                        flag = False
                        CLASS = x.text.split(" ")[1].strip("\t\n")
                        #print("CLASS = ",CLASS)
                        cs.append(CLASS)
                        
                    elif "申請人/商標/標章權人" ==  x.text:
                        #print("A<-"+str(x.text))
                        flaga = True
                        flag = False
                    elif "代理人" ==  x.text:
                        #print("B<-"+str(x.text))
                        flagb = True
                        flag = False
                    elif 'colTitle' in x.attrs['class']:
                            #print("<-"+x.text)
                            L.append(x.text)
                            flag = False
                    else:
                        #print("\t->"+str(x.text))
                        flag = True

                    if flag == True:
                        D[L[-1]] = x.text
            except:
                continue
                #print("")
                #print(x.text)
                #print("")
        
        
        index  = html_text.find('<td class="col-xs-3 colTitle">商標樣態／圖樣顏色</td>')
        sub = html_text[index:index+85]
        sub = sub.split("<td>")[-1].split("</td>")
        color = sub[0].strip("\t\n")
        #print(color)
        D['商標樣態／圖樣顏色'] = color
        
        '''
        print(A)
        print(B)
        print(C)
        print(D)
        '''
        
        os.chdir('/home/jack/3_HTML to CSV')
        return A
A = get_Content(107071791)
#print(A)

In [3]:
X = collections.OrderedDict()
print(X == {})
print(A)

True
OrderedDict([('1申請人/商標/標章權人_中文名稱', '瑞典商瑞典健身貿易股份有限公司'), ('2申請人/商標/標章權人_中文地址', '瑞典'), ('3申請人/商標/標章權人_英文名稱', 'SWEDISH FITNESS TRADING AKTIEBOLAG'), ('4申請人/商標/標章權人_國籍', '瑞典')])


In [6]:
def runhtmltocsv(failname):
    
    cnt = 0
    dircnt = 0
    fail = []
    #DF = pd.DataFrame()

    os.chdir('/home/jack/2_Reject/html')
    dirs = os.listdir()
    
    for j in dirs:
        os.chdir('/home/jack/2_Reject/html/%s'%str(j))
        files = os.listdir()
        dircnt += 1
       
        for i in files:
            if '.html' not in i:
                continue
            else:
                name = i.split(".")[0]
                #print(name)
                try:
                    if len(fail) > 30:
                    #if cnt == 5:
                        break
                    A = get_Content(int(name))
                    
                    #new = pd.DataFrame([D])
                    #DF = DF.append(new,ignore_index=True,sort=True)
                    if A == {}:
                        cnt += 1
                        print(name)
                        fail.append(name)
                except:
                    #fail.append(name)
                    print("?_?")
        goback3()
        with open('CSV Progress3.txt','a+') as t:
            t.write("CSV Progress3: "+str(dircnt)+" / 40; "+str(cnt)+" / 80597, Failed: "+str(len(fail))+"\n")
        print("CSV Progress3: "+str(dircnt)+" / 40; "+str(cnt)+" / 80597, Failed: "+str(len(fail))+"\n")
        
        if len(fail) > 30:
        #if cnt == 5:
            break 
        
    goback3()
    #DF.to_csv("RejectContent.csv")
    if len(fail) > 0:
        with open('%s.csv'%failname,'a+' ,newline='') as fd:
            writer = csv.writer(fd)
            writer.writerow(['caseno'])
            for k in fail:
                writer.writerow([k])
        with open('CSV Progress3.txt','a+') as t:
            for i in fail:
                t.write("Fail: "+str(i)+"\n")
def goback2():
    os.chdir('/home/jack/2_Reject')
def goback3():
    os.chdir('/home/jack/3_HTML to CSV')     

In [7]:
runhtmltocsv("fuck")

CSV Progress3: 1 / 40; 0 / 80597, Failed: 0

CSV Progress3: 2 / 40; 0 / 80597, Failed: 0

CSV Progress3: 3 / 40; 0 / 80597, Failed: 0

CSV Progress3: 4 / 40; 0 / 80597, Failed: 0

CSV Progress3: 5 / 40; 0 / 80597, Failed: 0

CSV Progress3: 6 / 40; 0 / 80597, Failed: 0

CSV Progress3: 7 / 40; 0 / 80597, Failed: 0

CSV Progress3: 8 / 40; 0 / 80597, Failed: 0

CSV Progress3: 9 / 40; 0 / 80597, Failed: 0

CSV Progress3: 10 / 40; 0 / 80597, Failed: 0

CSV Progress3: 11 / 40; 0 / 80597, Failed: 0

CSV Progress3: 12 / 40; 0 / 80597, Failed: 0

CSV Progress3: 13 / 40; 0 / 80597, Failed: 0

106062528
CSV Progress3: 14 / 40; 1 / 80597, Failed: 1

CSV Progress3: 15 / 40; 1 / 80597, Failed: 1

CSV Progress3: 16 / 40; 1 / 80597, Failed: 1

CSV Progress3: 17 / 40; 1 / 80597, Failed: 1

CSV Progress3: 18 / 40; 1 / 80597, Failed: 1

CSV Progress3: 19 / 40; 1 / 80597, Failed: 1

CSV Progress3: 20 / 40; 1 / 80597, Failed: 1

CSV Progress3: 21 / 40; 1 / 80597, Failed: 1

CSV Progress3: 22 / 40; 1 / 80597